# Get the data

In [2]:
import pickle
import os
import sys

In [3]:
# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory (NotesParser.py is one level up)
parent_dir = os.path.dirname(current_dir)

# Add the parent directory to the system path
sys.path.append(parent_dir)

from NotesParser import NotesParser

In [4]:
notes = []
RESO_PATH = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
DATA_PATH = os.path.join(RESO_PATH, 'data', 'Chopin')
NOTES_PATH = os.path.join(DATA_PATH, 'notes')

# If notes have already been dumped, load them, else run parser
if os.path.exists(NOTES_PATH):
    with open(NOTES_PATH, 'rb') as notes_file:
        notes = pickle.load(notes_file)
else:
    parser = NotesParser(DATA_PATH)
    notes = parser.get_notes_from_files()
    
    with open(NOTES_PATH, 'wb') as notes_file:
        pickle.dump(notes, notes_file)

In [5]:
parser = NotesParser()
notes = parser.get_notes_from_files(path=DATA_PATH, concatFiles=False)

Parsing 102 files from Chopin
1/102 Chopin, Frédéric, 2 Mazurkas, B.16, NtzP_wuR1iQ.mid
2/102 Chopin, Frédéric, Barcarolle, Op.60, UU21X-wmD0Q.mid
3/102 Chopin, Frédéric, Hexaméron Variation No.6, upfyIUP6U9Y.mid
4/102 Chopin, Frédéric, Mazurka in F-sharp major, fxp3IQ6pxYs.mid
5/102 Chopin, Frédéric, Mazurkas, Op.68, -2wScu6HtcY.mid
6/102 Chopin, Frédéric, Nocturnes, Op.55, e3yrEEM5j_s.mid
7/102 Chopin, Frédéric, Polonaise-fantaisie, Op.61, 177qJoCI9Zw.mid
8/102 Chopin, Frédéric, Scherzo No.3, Op.39, rjmUbZ3Aumw.mid
9/102 Chopin, Frédéric, Impromptu No.1, Op.29, baZGdj0xLYg.mid
10/102 Chopin, Frédéric, 3 Ecossaises, Op.72 No.3, z6h_Pj9qGLY.mid
11/102 Chopin, Frédéric, Nocturnes, Op.62, 1FeogxNAJWY.mid
12/102 Chopin, Frédéric, Mazurkas, Op.17, yU6jtMCYCQs.mid
13/102 Chopin, Frédéric, Mazurkas, Op.7, FsFDT24JElU.mid
14/102 Chopin, Frédéric, Scherzo No.4, Op.54, 9CNYX7OkceA.mid
15/102 Chopin, Frédéric, Polonaises, Op.26, Pp4v42suFQg.mid
16/102 Chopin, Frédéric, Berceuse, Op.57, FZtBwlxL

In [6]:
notes_parsed = [", ".join(x for x in song)[0:-1] for song in notes]

In [8]:
import json
with open('notes_parsed.json', 'w') as f:
    json.dump(notes_parsed, f)

# Set up the model

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation
from keras.utils import to_categorical
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint
from music21 import converter, note, chord
import numpy as np

### Define hyperparameters

In [ ]:
SEQUENCE_LENGTH = 100
EPOCHS = 1  # Adjust as needed
BATCH_SIZE = 128  # Adjust as needed

### Prepare sequences for LSTM

In [ ]:
# Create input sequences and corresponding output
unique_notes = sorted(set(notes))
note_to_int = dict((note, number) for number, note in enumerate(unique_notes))
int_to_note = dict((number, note) for number, note in enumerate(unique_notes))

input_sequences = []
output_sequences = []

for i in range(len(notes) - SEQUENCE_LENGTH):
    sequence_in = notes[i:i + SEQUENCE_LENGTH]
    sequence_out = notes[i + SEQUENCE_LENGTH]
    input_sequences.append([note_to_int[char] for char in sequence_in])
    output_sequences.append(note_to_int[sequence_out])

In [ ]:
print(len(unique_notes))

### Configure the model

In [ ]:
# Reshape input sequences
x = np.reshape(input_sequences, (len(input_sequences), SEQUENCE_LENGTH, 1))
x = x / float(len(unique_notes))

# One-hot encode output sequences
y = to_categorical(output_sequences)

model = Sequential()
model.add(LSTM(512, input_shape=(x.shape[1], x.shape[2]), return_sequences=True))
model.add(LSTM(512, return_sequences=True))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

check_point_path = "model/cp_{epoch:04d}.keras"
check_point = ModelCheckpoint(filepath=check_point_path, save_best_only=True, monitor='loss', save_freq='epoch')


### Train the model

In [ ]:
model.fit(x, y, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[check_point])

In [ ]:
import importlib
import generate
importlib.reload(generate)
generate.generate(model, input_sequences, int_to_note, unique_notes, "out.mid", 200)